# Firebase

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Khởi tạo kết nối với Firebase
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://coffee-4053c-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [ ]:
def GetProducts():
    # Lấy dữ liệu từ nút "SanPham" trong Firebase
    product_data = db.reference('SanPham').get()

    # Lấy dữ liệu từ nút "LoaiSanPham" trong Firebase
    product_type_data = db.reference('LoaiSanPham').get()
    list_product_type = list(product_type_data.values())

    # Xử lý dữ liệu
    products = [
        {
            'MaSanPham': product['MaSanPham'],
            'TenSanPham': product['TenSanPham'],
            'SoLuong': product['SoLuong'],
            'MoTa': product['Mota'],
            'HinhAnh': product['HinhAnh'],
            'LoaiSanPham': next((x['LoaiSanPham'] for x in list_product_type if x['MaLoaiSanPham'] == product['MaLoaiSanPham']), None),
            'CongThuc': list(product['CongThuc'].values())
        }
        for product in product_data.values()
    ]

    return products

In [ ]:
from collections import Counter

def GetTopProductID(number = 5):
  bill_data = db.reference('HoaDon').get()

  product_quantities = Counter()

  for bill_details in bill_data.values():
      for product_details in bill_details['ChiTietHoaDon'].values():
          product_quantities[product_details['MaSanPham']] += product_details['SoLuong']

  top_products = product_quantities.most_common(number)

  return [productID for productID, quantity in top_products]

In [ ]:
products = GetProducts()
product_size = len(products)

In [ ]:
products[:5]

[{'MaSanPham': 'SP0001',
  'TenSanPham': 'Cà phê sữa',
  'SoLuong': 18,
  'MoTa': 'Cà phê sữa siêu ngon',
  'HinhAnh': 'https://res.cloudinary.com/dev9hnuhw/image/upload/v1717966721/coffee/vigdytrefgtuwev3wsnx.jpg',
  'LoaiSanPham': 'Cà phê',
  'CongThuc': [{'MaDonVi': 'DV0002',
    'MaNguyenLieu': 'NL0003',
    'SoLuong': 20,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Cà phê'},
   {'MaDonVi': 'DV0004',
    'MaNguyenLieu': 'NL0006',
    'SoLuong': 200,
    'TenDonVi': 'ml',
    'TenNguyenLieu': 'Sữa'}]},
 {'MaSanPham': 'SP0002',
  'TenSanPham': 'Americano đá',
  'SoLuong': 5,
  'MoTa': 'Americano đá siêu ngon',
  'HinhAnh': 'https://res.cloudinary.com/dev9hnuhw/image/upload/v1715354979/coffee/u1tgayzvetmsbihacuhp.png',
  'LoaiSanPham': 'Cà phê',
  'CongThuc': [{'MaDonVi': 'DV0001',
    'MaNguyenLieu': 'NL0003',
    'SoLuong': 10,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Cà phê'}]},
 {'MaSanPham': 'SP0003',
  'TenSanPham': 'Latte đá',
  'SoLuong': 10,
  'MoTa': 'Latte đá siêu ngon',
 

In [ ]:
def GetIngredients():
    ingredient_data = db.reference('NguyenLieu').get()
    ingredients = list(ingredient_data.values())

    return ingredients

In [ ]:
ingredients = GetIngredients()

In [ ]:
ingredients

[{'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0001',
  'SoLuong': 34,
  'TenNguyenLieu': 'Muối'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0002',
  'SoLuong': 38,
  'TenNguyenLieu': 'Đường'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0003',
  'SoLuong': 8.8,
  'TenDonVi': 'Kg',
  'TenNguyenLieu': 'Cà phê'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0004',
  'SoLuong': 10,
  'TenDonVi': 'Kg',
  'TenNguyenLieu': 'Ca cao'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0005',
  'SoLuong': 10,
  'TenNguyenLieu': 'Bột quế'},
 {'MaDonVi': 'DV0003',
  'MaNguyenLieu': 'NL0006',
  'SoLuong': 7,
  'TenDonVi': 'l',
  'TenNguyenLieu': 'Sữa'},
 {'MaDonVi': 'DV0003',
  'MaNguyenLieu': 'NL0007',
  'SoLuong': 10,
  'TenNguyenLieu': 'Trà đen'},
 {'MaDonVi': 'DV0001',
  'MaNguyenLieu': 'NL0008',
  'SoLuong': 10,
  'TenNguyenLieu': 'Cà phê hoà tan'},
 {'MaDonVi': 'DV0004',
  'MaNguyenLieu': 'NL0009',
  'SoLuong': 10000,
  'TenDonVi': 'ml',
  'TenNguyenLieu': 'Kem tươi'},
 {'MaDonVi': 'DV0003',
  'MaNg

# Chatbot

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.layers import Input, Lambda, Dense, Concatenate, Conv1D, GlobalAveragePooling1D, Dropout, BatchNormalization, Bidirectional, LSTM, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/tensorflow/python/types/core.py:79: RuntimeWarning: coroutine 'Server.serve' was never awaited
  class Value(Tensor):


In [ ]:
# Configuration and Model Parameters
bert_name = "vinai/phobert-base"

# Load pre-trained BERT model and tokenizer
bert = TFAutoModel.from_pretrained(bert_name)
tokenizer = AutoTokenizer.from_pretrained(bert_name)

MAX_LEN_CONTEXT = 25

class BERTEmbeddingLayer(Layer):
    def __init__(self, bert_model_name, max_length, **kwargs):
        super(BERTEmbeddingLayer, self).__init__(**kwargs)
        self.bert_model_name = bert_model_name
        self.max_length = max_length
        # Load the BERT model inside the layer
        self.bert = TFAutoModel.from_pretrained(bert_model_name)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        # Obtain BERT outputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Equivalent to outputs[0]

    def get_config(self):
        config = super(BERTEmbeddingLayer, self).get_config()
        config.update({
            'bert_model_name': self.bert_model_name,
            'max_length': self.max_length,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = tf.keras.models.load_model('model_coffee_question_classification.keras', custom_objects={'BERTEmbeddingLayer': BERTEmbeddingLayer})

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
MAX_LEN_CONTEXT = 25

def getNumberPredict(question):
  test_contexts = [question]

  # Tokenize the test contexts
  test_encodings = tokenizer(
      test_contexts,
      truncation=True,
      padding='max_length',
      max_length=MAX_LEN_CONTEXT,
      add_special_tokens=True,
      return_tensors='tf'
  )

  # Prepare input tensors
  test_input_ids = test_encodings['input_ids']
  test_attention_mask = test_encodings['attention_mask']

  # Make predictions using the model
  predictions = model.predict([test_input_ids, test_attention_mask], batch_size=32)

  # Convert predictions to class labels (e.g., using argmax)
  predicted_classes = np.argmax(predictions, axis=1)

  return predicted_classes[0]

  # If you have a dictionary mapping from class indices to class labels, apply it
  # Example:
  label_mapping = {0: 'infomation', 1: 'productTopSell', 2: 'recommendedProduct', 3: 'ingredient', 4: 'quality', 5: 'greeting'}
  predicted_labels = [label_mapping.get(pred, 'unknown') for pred in predicted_classes]

  # Print the predicted labels
  for context, label in zip(test_contexts, predicted_labels):
      print(f"Question: {context} --> Predicted Label: {label}")

# DSA

In [ ]:
# KMP algorithm for searching product names in a precomputed LPS of a question
def compute_lps_array(pattern):
    lps = [0] * len(pattern)
    length = 0  # Length of the previous longest prefix suffix
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def kmp_search_with_precomputed_lps(text, pattern, lps):
    i = 0  # Index for text
    j = 0  # Index for pattern

    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1

        if j == len(pattern):
            return True  # Pattern found
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return False  # Pattern not found

# Function to find product names in a question using precomputed LPS
def find_products_in_question_kmp_precomputed(question, product_list):
    question_lower = question.lower()
    lps_question = compute_lps_array(question_lower)

    detected_products = []

    for product in product_list:
        product_name = product["TenSanPham"].lower()
        if kmp_search_with_precomputed_lps(question_lower, product_name, lps_question):
            detected_products.append(product)

    return detected_products

def find_ingredients_in_question_kmp_precomputed(question, ingredient_list):
    question_lower = question.lower()
    lps_question = compute_lps_array(question_lower)

    detected_ingredients = []

    for ingredient in ingredient_list:
        ingredient_name = ingredient["TenNguyenLieu"].lower()
        if kmp_search_with_precomputed_lps(question_lower, ingredient_name, lps_question):
            detected_ingredients.append(ingredient)

    return detected_ingredients

In [ ]:
def getProductFromIngredients(products, ingredients):
    ingredient_codes = {ingredient["MaNguyenLieu"] for ingredient in ingredients}

    matching_products = []

    for product in products:
        product_ingredient_codes = {ingredient['MaNguyenLieu'] for ingredient in product['CongThuc']}

        if ingredient_codes.issubset(product_ingredient_codes):
            matching_products.append(product)

    return matching_products[:5]

In [ ]:
question = "đường"

x = find_ingredients_in_question_kmp_precomputed(question, ingredients)
y = getProductFromIngredients(products, x)
y

[{'MaSanPham': 'SP0051',
  'TenSanPham': 'Cà phê lạnh chua',
  'SoLuong': 0,
  'MoTa': 'cà phê ngon',
  'HinhAnh': 'https://res.cloudinary.com/dev9hnuhw/image/upload/v1717966829/coffee/djvuxrsexlbrq9nyixtj.jpg',
  'LoaiSanPham': 'Cà phê',
  'CongThuc': [{'MaDonVi': 'DV0001',
    'MaNguyenLieu': 'NL0002',
    'SoLuong': 100,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Đường'},
   {'MaDonVi': 'DV0001',
    'MaNguyenLieu': 'NL0004',
    'SoLuong': 10,
    'TenDonVi': 'g',
    'TenNguyenLieu': 'Ca cao'}]}]

In [ ]:
def binary_search(product_id):
    left, right = 0, product_size - 1
    while left <= right:
        mid = (left + right) // 2
        mid_id = extract_number_from_id(products[mid]['MaSanPham'])
        target_id = extract_number_from_id(product_id)
        if mid_id == target_id:
            return products[mid]
        elif mid_id < target_id:
            left = mid + 1
        else:
            right = mid - 1
    return None  # Không tìm thấy sản phẩm

## Chatbot

In [ ]:
def extract_number_from_id(product_id):
    # Lấy phần số từ mã sản phẩm dạng "SP0001"
    return int(product_id[2:])

In [ ]:
def getInfomationProduct(product):
    # Tạo câu mô tả sản phẩm
    text = f"{product['TenSanPham']} thuộc loại sản phẩm {product['LoaiSanPham']}. "
    text += f"Sản phẩm này được mô tả như sau: {product['MoTa']}. "
    text += "Nguyên liệu chính gồm: "

    # Danh sách các nguyên liệu từ 'CongThuc'
    ingredients = ", ".join(congthuc.get('TenNguyenLieu', 'N/A') for congthuc in product['CongThuc'])
    text += f"{ingredients}. "

    # Thông tin về số lượng sản phẩm hiện có
    text += f"Số lượng hiện tại trong kho: {product['SoLuong']}.\n"

    return text

In [ ]:
import random

def get_quality_info(products):
    product = ""

    if len(products) == 1:
        product = products[0]
    else:
        product = random.choice(products)

    message = f"Sản phẩm {product['TenSanPham']} rất ngon "
    ingredients = ", ".join(congthuc.get('TenNguyenLieu', 'N/A') for congthuc in product['CongThuc'])
    message += f"vì được làm từ những nguyên liệu chất lượng như {ingredients}."

    return message

In [ ]:
import requests

url = "http://huynhnhan2003.pythonanywhere.com/recommend"

In [ ]:
beginMessage = "Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo."
endMessage = (
    "Mong bạn hài lòng với câu trả lời của chúng tôi.\n"
    "Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, "
    "sản phẩm bán chạy và gợi ý sản phẩm.\nVí dụ:\n"
    "- Cà phê sữa còn hàng không?\n"
    "- Cho tôi danh sách sản phẩm bán chạy của quán.\n"
    "- Gợi ý cho tôi sản phẩm với.\n"
    "Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán."
)

In [ ]:
def getMessage(number, question, customer_id):
    message = [beginMessage]

    # Find products in the question
    productInQuestion = find_products_in_question_kmp_precomputed(question, products)
    sizeProductInQuestion = len(productInQuestion)

    ingredientInQuestion = find_ingredients_in_question_kmp_precomputed(question, ingredients)

    if number == 0:
        # Information
        message.append("\n")
        message.extend(getInfomationProduct(product) for product in productInQuestion)

    elif number == 1:
        # Top sell
        productIDs = GetTopProductID()
        productTopSell = [binary_search(productID) for productID in productIDs]

        message.append('Sản phẩm được bán chạy nhiều nhất của cửa hàng:\n')
        message.extend(f'- {product["TenSanPham"]}\n' for product in productTopSell)

    elif number == 2:
        # Recommendation
        payload = {
            "MaSanPham": productInQuestion[0]['MaSanPham'] if sizeProductInQuestion != 0 else "SP0001",
            "MaKhachHang": "" if sizeProductInQuestion != 0 else customer_id
        }

        response = requests.post(url, json=payload)
        productsRecommend = response.json()

        message.append('Sản phẩm gợi ý cho bạn:\n')
        message.extend(f'- {product["TenSanPham"]}' for product in productsRecommend)

    elif number == 3:
        productFromIngredients = getProductFromIngredients(products, ingredientInQuestion)
        message.append('Các sản phẩm có thành phần bạn quan tâm:\n')
        message.extend(f'- {getInfomationProduct(product)}' for product in productFromIngredients)

    elif number == 4:
        # Quality information
        message.append(f'Thông tin về chất lượng của sản phẩm: {get_quality_info(productInQuestion)}\n')

    elif number == 5:
        # Greeting
        message = ['Chào bạn! Cảm ơn bạn đã quan tâm đến cửa hàng của chúng tôi. Nếu bạn cần thông tin cụ thể về sản phẩm hoặc dịch vụ, vui lòng hỏi nhé!']

    # Combine the messages into a single string
    return '\n'.join(message) + endMessage

# API

In [ ]:
def chatBotGetMessage(question, customer_id):
  number = getNumberPredict(question)
  return getMessage(number, question, customer_id)

In [ ]:
print(chatBotGetMessage("tôi nên dùng cà phê nào", "KH0001"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo.
Sản phẩm gợi ý cho bạn:

- Cà phê Cappuccino

- Cà phê Espresso

- Cà phê Mocha

- Cà phê kem tươi

- Cà phê lạnh chua
Mong bạn hài lòng với câu trả lời của chúng tôi.
Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, sản phẩm bán chạy và gợi ý sản phẩm.
Ví dụ:
- Cà phê sữa còn hàng không?
- Cho tôi danh sách sản phẩm bán chạy của quán.
- Gợi ý cho tôi sản phẩm với.
Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán.


In [ ]:
!pip install fastapi
!pip install pydantic
!pip install pyngrok
!pip install requests
!pip install uvicorn

In [ ]:
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
import json
import uvicorn
import nest_asyncio
from pyngrok import ngrok

In [ ]:
nest_asyncio.apply()

In [ ]:
app = FastAPI()

@app.get("/")
async def home():
  return {"Hello API Coffee"}

In [ ]:
from pydantic import BaseModel

class requestChatbot(BaseModel):
    MaKhachHang : str
    CauHoi : str

In [ ]:
@app.post("/chatbot")
async def reply(request: requestChatbot):
    try:
        # Get JSON data from the request
        request_data = await request.json()
        user_id = request_data['MaKhachHang']
        question = request_data['CauHoi']

        # Generate the chatbot's response
        message = chatBotGetMessage(question, user_id)

        return {"message": message}
    except Exception as e:
        return {"Message": "error", "Details": str(e)}

In [ ]:
!ngrok config add-authtoken 2RTXfbCm3Gm5DCFEFfHAprwXgdI_3HPFH8ExdVKq4631Gixxp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
public_url = ngrok.connect(8050, bind_tls=True)
 # Specify the protocol and port correctly
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://1554-34-91-47-113.ngrok-free.app" -> "http://localhost:8050"


In [ ]:
import os
import threading
from flask import Flask, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
public_url = ngrok.connect(5000).public_url
print(public_url)

Exception in thread Thread-14 (new_run):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: run_with_ngrok.<locals>.new_run() got an unexpected keyword argument 'use_reloader'
